Notes for the near-term "Can we match some secondary service request to a service request in the previous harvests?"

Maybe. Likely. We'll see.

ogc should be straightforward.
ignore thredds for now.
oai-pmh also okay.
not actually sure about opensearch.

the preprocessing:
    put together the set of base urls as a rough cut (if bases don't match, bail)
    for base urls that match, check the expected query parameters
    if match, relate as parent service.
    if not, bail.

In [3]:
import os
import csv
import json
import urllib, urlparse

In [ ]:
# with open('harvest_baseurls.txt', 'r') as f:
#     base_urls = [a.strip() for a in f.readlines()]

In [10]:
with open('harvest_urls_by_protocol.txt', 'r') as f:
    response_urls = [a.replace('"', '').strip().split('||') for a in f.readlines()[1:]]

In [11]:
print response_urls[:3]

[['http://107.20.228.18/arcgis/services/Wetlands/MapServer/WMSServer?request=GetCapabilities&service=WMS', 'OGC:WMS'], ['http://10ncee.org/?format=feed&type=atom', 'ATOM'], ['http://10ncee.org/?format=feed&type=rss', 'ATOM']]


In [18]:
nesting = {}
for source_url, protocol in response_urls:
    if protocol == 'DDI':
        continue
    parts = urlparse.urlparse(source_url)
    
    bases = nesting.get(parts.netloc, [])
    params = dict( (k, v if len(v)>1 else v[0] ) 
           for k, v in urlparse.parse_qs(parts.query).iteritems() )
    
    bases.append((protocol, source_url, params))
    nesting[parts.netloc] = bases

# print [nesting[k] for k in nesting.keys()[:10]]

In [19]:
potentials = {k:v for k, v in nesting.iteritems() if len(v) > 1}

In [20]:
[nesting[k] for k in potentials.keys()[:10]]

[[('ATOM', 'http://photojournal.jpl.nasa.gov/rss/gallery/universe', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/mission/SRTM', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/target/Earth', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/target/Other', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/target/Sun', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Earth', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Jupiter', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Mars', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Mercury', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Neptune', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Saturn', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Uranus', {}),
  ('ATOM', 'http://photojournal.jpl.nasa.gov/rss/targetFamily/Venus', {})],
 [('', 'http://www.icpsr.umich.edu/OLC/xml/cl

In [ ]:
for k, v in potentials.iteritems():
    # so. go through the dict to match
    # certain param sets in the lists